<a href="https://colab.research.google.com/github/Loop3D/2022_iamg_workshop/blob/main/draw_your_own.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing dependencies
This notebook will use two libraries from the Loop project
1. LoopStructural - https://github.com/Loop3D/LoopStructural
2. map2loop - https://github.com/Loop3D/map2loop-2


The following code blocks will install the required dependencies for the Loop libraries into this colab environment. Alternatively, the workshop can be run using the docker image.

## map2loop + LoopStructural

In [ ]:
!pip install rasterio
!pip install git+https://github.com/geopandas/geopandas.git@v0.10.2
!pip install hjson
!pip install owslib
!pip install git+https://github.com/Loop3D/map2model_cpp.git
!pip install git+https://github.com/Loop3D/LoopProjectFile.git
!pip install pygeos
!pip install mplstereonet
!pip install lavavu-osmesa
!pip install git+https://github.com/Loop3d/LoopStructural
!pip install git+https://github.com/Loop3D/map2loop-2.git
!pip install beartype
!pip install git+https://github.com/giswqs/leafmap.git


In [ ]:

import leafmap
import pandas as pd
import json
import random
from shapely.geometry import Polygon
import ipywidgets as widgets
import geopandas as gpd
import os
import hjson
from map2loop.m2l_enums import VerboseLevel
from map2loop.project import Project
import random
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer


In [ ]:
center=(-22.6,117.3)

m =leafmap.Map( center=center, zoom=8,scroll_wheel_zoom=True)
m.add_basemap( basemap='OpenTopoMap')

m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='2_5m_interpgeop15_4326',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Geology data from GSWA',
    name='geology',
    shown=True)
aurl='https://www.loopwms.xyz/geoserver/loop/wms?'
m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
#m.add_wms_layer(url='https://13.211.217.129:8080/geoserver/loop/wms?',
#    layers='2_5m_interpstrucl15_4326',
    layers='2_5m_interpstrucl15_4326',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Linear features data from GSWA',
    name='faults/folds')

m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='waroxi_wa_4326_bed',
    format='image/png',
    transparent=True,
    attribution='Outcrop data from GSWA',
    name='outcrops')

m

In [ ]:

rect=m.draw_features

if(len(rect)==0):
  minlong=117.13698
  maxlong=117.564464
  minlat= -22.690712
  maxlat=-22.396454

else:
  minlong=rect[0]['geometry']['coordinates'][0][0][0]
  maxlong=rect[0]['geometry']['coordinates'][0][2][0]
  minlat= rect[0]['geometry']['coordinates'][0][0][1]
  maxlat= rect[0]['geometry']['coordinates'][0][1][1]

In [ ]:
src_crs = "epsg:4326"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
dst_crs = "epsg:28350" # coordinate system for example data


use_roi_clip=False
roi_clip_path=''
    

bounds=(minlong,maxlong,minlat,maxlat)


lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
mbbox = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 
bbox=mbbox.total_bounds
st_bbox=[bbox[0],bbox[1],bbox[2],bbox[3]]
print(src_crs,mbbox.total_bounds)
mbbox=mbbox.to_crs(dst_crs)
print(dst_crs,mbbox.total_bounds)

In [ ]:
savepath=str(random.randint(1,10000))
proj = Project( 
                 loopdata_state = "WA",
            project_path=savepath,
    project_crs='EPSG:28350',
    verbose_level=VerboseLevel.NONE,
    working_projection='EPSG:28350',
                )

proj.update_config(
                    out_dir=savepath,
                    bbox_3d={
                         "minx": mbbox.total_bounds[0], #500000,
                         "miny": mbbox.total_bounds[1], #7490000,
                         "maxx": mbbox.total_bounds[2], #545000,
                         "maxy": mbbox.total_bounds[3], #7520000,
                         "base": -4800,
                         "top": 1200,
                    },
                    project_crs='EPSG:28350',

                  )


In [ ]:
proj.run()

In [ ]:
minx,miny,maxx,maxy = proj.config.bbox
model_base = proj.config.bbox_3d['base']
model_top = proj.config.bbox_3d['top']

In [ ]:
f=open(proj.config.tmp_path+'/bbox.csv','w')
f.write('minx,miny,maxx,maxy,lower,upper\n')
ostr='{},{},{},{},{},{}\n'.format(minx,miny,maxx,maxy,model_base,model_top)
f.write(ostr)
f.close()

In [ ]:
fault_params = {'interpolatortype':'FDI',
                'nelements':1e4,
                'step':10,
                'fault_buffer':0.2,

               }
foliation_params = {'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
                    'nelements':5e4,  # how many tetras/voxels
                    'buffer':2.,  # how much to extend nterpolation around box

                   }


model, m2l_data = GeologicalModel.from_map2loop_directory(proj.config.project_path,
                                                          evaluate=False,
                                                          fault_params=fault_params,
                                                          rescale=False,
                                                          foliation_params=foliation_params)

In [ ]:
model.update()

In [ ]:
view = LavaVuModelViewer(model)
view.add_model_surfaces()
view.interactive()

In [ ]:
 view.display()